In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')

2024-03-23 13:37:50.447677: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 13:37:50.471990: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 13:37:50.472016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 13:37:50.472042: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 13:37:50.477664: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-23 13:37:52.184972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 13:37:52.207409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 13:37:52.207625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 299, 299
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 25744 files belonging to 167 classes.
Using 18021 files for training.


2024-03-23 13:37:52.753778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 13:37:52.754080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-23 13:37:52.754244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


In [4]:
preprocess_input = tf.keras.applications.inception_v3.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)

input_1
conv2d
batch_normalization
activation
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
max_pooling2d
conv2d_3
batch_normalization_3
activation_3
conv2d_4
batch_normalization_4
activation_4
max_pooling2d_1
conv2d_8
batch_normalization_8
activation_8
conv2d_6
conv2d_9
batch_normalization_6
batch_normalization_9
activation_6
activation_9
average_pooling2d
conv2d_5
conv2d_7
conv2d_10
conv2d_11
batch_normalization_5
batch_normalization_7
batch_normalization_10
batch_normalization_11
activation_5
activation_7
activation_10
activation_11
mixed0
conv2d_15
batch_normalization_15
activation_15
conv2d_13
conv2d_16
batch_normalization_13
batch_normalization_16
activation_13
activation_16
average_pooling2d_1
conv2d_12
conv2d_14
conv2d_17
conv2d_18
batch_normalization_12
batch_normalization_14
batch_normalization_17
batch_normalization_18
activation_12
activation_14
activation_17
activation_18
mixed1
conv2d_22
batch_normalization_22
activation_22
conv2d

In [6]:
from keras.src import regularizers

inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32, kernel_regularizer=regularizers.l2(0.005))(x)

model1 = tf.keras.Model(inputs, outputs)

In [7]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                             

In [8]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 

In [9]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_inceptionv3.hdf5', verbose=1, save_best_only=True)

history_eff_b0 = model1.fit(train_dataset,  
                                        validation_data=validation_dataset,
                                        epochs=10,
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 1/10


2024-03-23 13:37:58.855448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-23 13:37:58.956851: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-23 13:38:00.584230: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6485183e0c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-23 13:38:00.584252: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-23 13:38:00.593217: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-23 13:38:00.662933: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


564/564 [==============================] - ETA: 0s - loss: 4.8180 - accuracy: 0.0218
Epoch 1: val_loss improved from inf to 4.39201, saving model to Best_models/best_inceptionv3.hdf5


/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


564/564 [==============================] - 45s 71ms/step - loss: 4.8180 - accuracy: 0.0218 - val_loss: 4.3920 - val_accuracy: 1.2948e-04 - lr: 0.0010
Epoch 2/10
563/564 [============================>.] - ETA: 0s - loss: 4.1576 - accuracy: 0.0354
Epoch 2: val_loss improved from 4.39201 to 3.88642, saving model to Best_models/best_inceptionv3.hdf5
564/564 [==============================] - 37s 66ms/step - loss: 4.1576 - accuracy: 0.0354 - val_loss: 3.8864 - val_accuracy: 0.0230 - lr: 0.0010
Epoch 3/10
563/564 [============================>.] - ETA: 0s - loss: 3.6971 - accuracy: 0.0703
Epoch 3: val_loss improved from 3.88642 to 3.47313, saving model to Best_models/best_inceptionv3.hdf5
564/564 [==============================] - 40s 71ms/step - loss: 3.6970 - accuracy: 0.0703 - val_loss: 3.4731 - val_accuracy: 0.1220 - lr: 0.0010
Epoch 4/10
564/564 [==============================] - ETA: 0s - loss: 3.3561 - accuracy: 0.1021
Epoch 4: val_loss improved from 3.47313 to 3.32065, saving model t

In [10]:

for layer in base_model.layers[-50:]:
    layer.trainable = True


In [11]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_1 False
conv2d False
batch_normalization False
activation False
conv2d_1 False
batch_normalization_1 False
activation_1 False
conv2d_2 False
batch_normalization_2 False
activation_2 False
max_pooling2d False
conv2d_3 False
batch_normalization_3 False
activation_3 False
conv2d_4 False
batch_normalization_4 False
activation_4 False
max_pooling2d_1 False
conv2d_8 False
batch_normalization_8 False
activation_8 False
conv2d_6 False
conv2d_9 False
batch_normalization_6 False
batch_normalization_9 False
activation_6 False
activation_9 False
average_pooling2d False
conv2d_5 False
conv2d_7 False
conv2d_10 False
conv2d_11 False
batch_normalization_5 False
batch_normalization_7 False
batch_normalization_10 False
batch_normalization_11 False
activation_5 False
activation_7 False
activation_10 False
activation_11 False
mixed0 False
conv2d_15 False
batch_normalization_15 False
activation_15 False
conv2d_13 False
conv2d_16 False
batch_normalization_13 False
batch_normalization_16 False
activati

In [12]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 10

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 11/25
563/564 [============================>.] - ETA: 0s - loss: 2.3589 - accuracy: 0.2712
Epoch 11: val_loss improved from 2.32850 to 2.24982, saving model to Best_models/best_inceptionv3.hdf5
564/564 [==============================] - 39s 63ms/step - loss: 2.3589 - accuracy: 0.2711 - val_loss: 2.2498 - val_accuracy: 0.3048 - lr: 0.0010
Epoch 12/25
564/564 [==============================] - ETA: 0s - loss: 2.2281 - accuracy: 0.2960
Epoch 12: val_loss improved from 2.24982 to 2.05184, saving model to Best_models/best_inceptionv3.hdf5
564/564 [==============================] - 35s 63ms/step - loss: 2.2281 - accuracy: 0.2960 - val_loss: 2.0518 - val_accuracy: 0.3145 - lr: 0.0010
Epoch 13/25
564/564 [==============================] - ETA: 0s - loss: 2.1407 - accuracy: 0.3096
Epoch 13: val_loss improved from 2.05184 to 2.01768, saving model to Best_models/best_inceptionv3.hdf5
564/564 [==============================] - 35s 62ms/step - loss: 2.1407 - accuracy: 0.3096 - val_loss: 2.017

In [13]:
len(base_model.layers)

311

In [17]:

for layer in base_model.layers[-290:]:
    layer.trainable = True


In [24]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.0000625),
                           metrics=['accuracy'])
start_epoch = 115

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+30,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 116/145
563/564 [============================>.] - ETA: 0s - loss: 0.2877 - accuracy: 0.9021
Epoch 116: val_loss did not improve from 0.59865
564/564 [==============================] - 39s 63ms/step - loss: 0.2878 - accuracy: 0.9020 - val_loss: 0.6235 - val_accuracy: 0.8795 - lr: 6.2500e-05
Epoch 117/145
564/564 [==============================] - ETA: 0s - loss: 0.2850 - accuracy: 0.9044
Epoch 117: val_loss did not improve from 0.59865
564/564 [==============================] - 35s 62ms/step - loss: 0.2850 - accuracy: 0.9044 - val_loss: 0.6349 - val_accuracy: 0.8826 - lr: 6.2500e-05
Epoch 118/145
563/564 [============================>.] - ETA: 0s - loss: 0.2875 - accuracy: 0.9029
Epoch 118: val_loss did not improve from 0.59865
564/564 [==============================] - 35s 63ms/step - loss: 0.2875 - accuracy: 0.9029 - val_loss: 0.6238 - val_accuracy: 0.8770 - lr: 6.2500e-05
Epoch 119/145
564/564 [==============================] - ETA: 0s - loss: 0.2867 - accuracy: 0.9024
Epoch 1

KeyboardInterrupt: 

In [25]:
loaded = tf.keras.models.load_model("Best_models/best_inceptionv3.hdf5")
loaded.evaluate(test_dataset)

361/361 [==============================] - 18s 47ms/step - loss: 0.9676 - accuracy: 0.8368


[0.9676294922828674, 0.8368124961853027]